# Extract ES cert from docker container
docker cp gdelt_elk-es01-1:/usr/share/elasticsearch/config/certs/ca/ca.crt ./ca.crt

In [1]:
from elasticsearch import Elasticsearch
from datetime import datetime
import os
CA_CERT_PATH = os.path.abspath("ca.crt")  # Converts to full path

# --- CONFIG ---
ES_HOST = "https://localhost:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "changeme"
ES_INDEX = "gkg"       

# Create the client with authentication
es = Elasticsearch(
    ES_HOST,
    basic_auth=(ES_USERNAME, ES_PASSWORD),
    verify_certs=False, # Set to False if you're using self-signed certs (not recommended for prod)
    ca_certs=CA_CERT_PATH
)

c:\Users\ngyee\AppData\Local\Programs\Python\Python312\Lib\site-packages\elasticsearch\_sync\client\__init__.py:403: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [2]:
print(CA_CERT_PATH)

c:\Users\ngyee\Projects\OTB stuff\GDELT_ELK\ca.crt


In [3]:
# --- INPUT ---
start_time = "2025-04-01 00:00:00"
end_time = "2025-04-02 23:59:00"

# --- Convert to RecordId format ---
def to_record_id_format(dt_str):
    return datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M%S")

record_id_start = to_record_id_format(start_time)
record_id_end = to_record_id_format(end_time)

# --- Query ---
query = {
    "_source": ["V2Persons.V1Person"],
    "query": {
        "range": {
            "RecordId": {
                "gte": record_id_start,
                "lte": record_id_end
            }
        }
    }
}

# --- Execute ---
response = es.search(index=ES_INDEX, body=query)  # Increase size if needed

print(response)

# --- Extract and Deduplicate Persons ---
entities = []
for hit in response["hits"]["hits"]:
    persons = hit["_source"].get("V2Persons", {}).get("V1Person", [])
    entities.extend(persons)

unique_entities = sorted(set(entities))

# --- Output ---
print(f"Entities found from {start_time} to {end_time}:")
for name in unique_entities:
    print("-", name)


{'took': 29, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'gkg', '_id': 'vTnB9ZUBL8Nog7K__d2v', '_score': 1.0, '_ignored': ['event.original.keyword', 'V2ExtrasXML.Links.keyword'], '_source': {'V2Persons': {'V1Person': ['Villa Rockstar', 'Spencer Matthews', 'Jamie Laing', 'Wendy Golledge', 'Sophie Habboo', 'Alison Perry']}}}, {'_index': 'gkg', '_id': 'vjnB9ZUBL8Nog7K__d2v', '_score': 1.0, '_ignored': ['event.original.keyword'], '_source': {'V2Persons': {'V1Person': ['Nicole Zhang', 'Wang Xiao Nan', 'Zeng Hao', 'Zixin Li']}}}, {'_index': 'gkg', '_id': 'vznB9ZUBL8Nog7K__d2v', '_score': 1.0, '_ignored': ['event.original.keyword', 'V2ExtrasXML.Links.keyword'], '_source': {}}, {'_index': 'gkg', '_id': 'wDnB9ZUBL8Nog7K__d2v', '_score': 1.0, '_ignored': ['event.original.keyword'], '_source': {'V2Persons': {'V1Person': ['Le Pew', 'Looney Tunescollector Vaultvo

c:\Users\ngyee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
